In [ ]:
{
    "cells": [
        {
            "cell_type": "markdown",
            "metadata": {},
            "source": [
                "# Import, publish and run projects in K8s clusters\n",
                "This notebook leverages the IOT ESP API to create and publish projects into ESP.\n",
                "\n",
                "Pre-requisite: This example will assume that at same level as this Jupyter Notebook there is an *xml_projects* folder with XML files representing proejcts the user wants to import and publish to ESP.\n",
                "\n",
                "In this notebook we will loop through all XML projects in xml_projects folder and do the following:\n",
                "1. Check if projects have already been imported\n",
                "2. If yes, import it using next version number. Otherwise import it normally with version 1 \n",
                "3. Make project public so all users can see it\n",
                "4. Publish project\n",
                "5. Synchronize projects from Studio to ESM\n",
                "6. Create ESM Deployment Cluster on which to run the projects\n",
                "7. Start projets on K8s cluster on ESM\n",
                "\n",
                "   \n",
                "Note: Please make sure you run [Imports and Global Variables](#imports) before executing anything else in this notebook. Also ensure you are authenticated by running [Get Access Token](#authentication).\n"
            ]
        },
        {
            "cell_type": "markdown",
            "metadata": {},
            "source": [
                "## Imports and Global Variables <a id='imports'></a>\n",
                "Run this cell before any of the others as it imports packages and sets variables that will be used throughout the notebook."
            ]
        },
        {
            "cell_type": "code",
            "execution_count": 55,
            "metadata": {},
            "outputs": [
                {
                    "name": "stdout",
                    "output_type": "stream",
                    "text": [
                        "Server: acme.kc4.ingress-nginx.espsc-kc5-m1.espstudio.sashq-d.openstack.sas.com\n",
                        "Username: fsduser\n",
                        "Password: Mercury7\n",
                        "ESM Deployment Cluster Name: test\n"
                    ]
                }
            ],
            "source": [
                "import requests\n",
                "import xml.etree.ElementTree as ET\n",
                "import os\n",
                "import json\n",
                "import time\n",
                "import sys\n",
                "from urllib3.exceptions import InsecureRequestWarning\n",
                "\n",
                "\n",
                "def bootstrap_server_and_credentials():\n",
                "    global server, username, password, chosen_deployment_name\n",
                "    server = \"acme.kc4.ingress-nginx.espsc-kc5-m1.espstudio.sashq-d.openstack.sas.com\"\n",
                "    username = \"fsduser\"\n",
                "    password = \"Mercury7\"\n",
                "    chosen_deployment_name = \"test\"\n",
                "    if len(sys.argv) > 3:\n",
                "        server = sys.argv[1]\n",
                "        username = sys.argv[2]\n",
                "        password = sys.argv[3]\n",
                "\n",
                "    print('Server: ' + server)\n",
                "    print('Username: ' + username)\n",
                "    print('Password: ' + password, flush=True)\n",
                "    print('ESM Deployment Cluster Name: ' + chosen_deployment_name, flush=True)\n",
                "\n",
                "# Suppress ssl warnings caused by verify=False\n",
                "requests.packages.urllib3.disable_warnings(category=InsecureRequestWarning)\n",
                "bootstrap_server_and_credentials()"
            ]
        },
        {
            "cell_type": "markdown",
            "metadata": {},
            "source": [
                "# Get Access token <a id='authentication'></a>"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": 56,
            "metadata": {},
            "outputs": [
                {
                    "name": "stdout",
                    "output_type": "stream",
                    "text": [
                        "eyJhbGciOiJSUzI1NiIsImprdSI6Imh0dHBzOi8vbG9jYWxob3N0L1NBU0xvZ29uL3Rva2VuX2tleXMiLCJraWQiOiJsZWdhY3ktdG9rZW4ta2V5IiwidHlwIjoiSldUIn0.eyJqdGkiOiI5ZTNjMWRlMWFlNDI0ZTgxYTBiMGE0YjRhNjYyNDhmNyIsImF1dGhvcml0aWVzIjpbIlZpeWFTQVNBZG1pbnMiLCJzdmlkZXZvcHMiLCJzdml2bXVzZXJzIiwiaW50Y2Fjb2Rlc2lnbmluZyIsInVuaXhfciZkIiwiVVNTQVMiLCJyY2ktY2lycnVzLWRldiIsImludGNhd2ViY2VydHJlcSIsIm9wZW5zdGFja3VzZXJzIiwicmNpLWNsdXN0ZXJkZXZtb2RlIiwiaW50Y2F1c2VycyJdLCJleHRfaWQiOiJjbj1STVMgRlNEIFByb2R1Y3RzIFVzZXIsb3U9R2VuZXJpYyBhbmQgU2hhcmVkIEFjY291bnRzLG91PUFkbWluLGRjPW5hLGRjPVNBUyxkYz1jb20iLCJzdWIiOiJlYTY1MTZlMi0wMTJiLTQ0MTktOWJhMS1kMTY3MWZmNjc2ZDAiLCJzY29wZSI6WyJvcGVuaWQiLCJ1YWEudXNlciJdLCJjbGllbnRfaWQiOiJzYXMuZWMiLCJjaWQiOiJzYXMuZWMiLCJhenAiOiJzYXMuZWMiLCJncmFudF90eXBlIjoicGFzc3dvcmQiLCJ1c2VyX2lkIjoiZWE2NTE2ZTItMDEyYi00NDE5LTliYTEtZDE2NzFmZjY3NmQwIiwib3JpZ2luIjoibGRhcCIsInVzZXJfbmFtZSI6ImZzZHVzZXIiLCJlbWFpbCI6ImZzZHVzZXJAdXNlci5mcm9tLmxkYXAuY2YiLCJhdXRoX3RpbWUiOjE3MTQwNTIwODAsInJldl9zaWciOiI5Njg4NWFlNiIsImlhdCI6MTcxNDA1MjA4MCwiZXhwIjoxNzE0MDU1NjgwLCJpc3MiOiJodHRwOi8vbG9jYWxob3N0L1NBU0xvZ29uL29hdXRoL3Rva2VuIiwiemlkIjoidWFhIiwiYXVkIjpbInVhYSIsIm9wZW5pZCIsInNhcy5lYyJdfQ.QY5XfQBtzrN2Wg_sKv3w17qvZWmPz6otR4z89gSC8ZkRppMXdLqaMLwpKLlCsBAznGokrKtVklnMC24JS2ocm-001zWrgl3Crjz0Nm8OmkmQJfyeNBoQfqz_krx3qRNvgLF6ziIDv8zZvS3_1t__9fDKxfFUQ0MQQWP0zAqwteSrqcOp2STyvVNIGf0fPF7HAHgp4xrUmMZb0XyZ4sakPioOoaXTKgumMpPpefU_P6v81HWJCAL_L_0folsv-YById-9KZXF4VadtR2E8t6awdLQ9zpFrjecpuAevO6CcV-9efGvYcb0RLxevUmJ-iEnx84aPiZP06Oldq0lWdvrmw\n"
                    ]
                }
            ],
            "source": [
                "def get_access_token():\n",
                "    body = {'grant_type': 'password', 'username': username, 'password': password}\n",
                "    headers = {'Content-Type': 'application/x-www-form-urlencoded', 'Authorization': 'Basic c2FzLmVjOg=='}\n",
                "    access_token_response = requests.post('http://' + server + '/SASLogon/oauth/token', data=body, headers=headers,\n",
                "                                          verify=False)\n",
                "    return access_token_response.json()[\"access_token\"]\n",
                "\n",
                "access_token = get_access_token()\n",
                "print(access_token)\n",
                "headers = {\"Content-Type\": \"application/json\", \"Authorization\": \"Bearer \" + access_token}"
            ]
        },
        {
            "cell_type": "markdown",
            "metadata": {},
            "source": [
                "# Function to get ESP Projects"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": 12,
            "metadata": {},
            "outputs": [],
            "source": [
                "def get_projects():\n",
                "    projects = []\n",
                "    get_projects_response = requests.get(\n",
                "        'http://' + server + '/SASEventStreamProcessingStudio/esp-project', headers=headers, verify=False)\n",
                "    if get_projects_response.status_code == 200:\n",
                "        projects = get_projects_response.json()[\"items\"]\n",
                "    return projects"
            ]
        },
        {
            "cell_type": "markdown",
            "metadata": {},
            "source": [
                "# Function to import project to ESP Studio"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": 13,
            "metadata": {},
            "outputs": [],
            "source": [
                "def import_project_to_studio(project_body):\n",
                "    project_id = \"\"\n",
                "    import_project_response = requests.post('http://' + server + '/SASEventStreamProcessingStudio/esp-project',\n",
                "                                            data=json.dumps(project_body), headers=headers, verify=False)\n",
                "    if import_project_response.status_code == 200:\n",
                "        project_id = import_project_response.json()[\"flowId\"]\n",
                "        print('success, project_id=' + project_id)\n",
                "    return project_id"
            ]
        },
        {
            "cell_type": "markdown",
            "metadata": {},
            "source": [
                "# Function to get next version number of a project"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": 14,
            "metadata": {},
            "outputs": [],
            "source": [
                "def get_next_project_version(project_id):\n",
                "    version = 2\n",
                "    get_next_version_response = requests.get(\n",
                "        'http://' + server + '/SASEventStreamProcessingStudio/project-versions/projects/' + project_id + '/nextVersion',\n",
                "        headers=headers, verify=False)\n",
                "    if get_next_version_response.status_code == 200:\n",
                "        version = get_next_version_response.json()[\"major\"]\n",
                "    else:\n",
                "        print('Failed to get next version')\n",
                "        print(get_next_version_response)\n",
                "    return version"
            ]
        },
        {
            "cell_type": "markdown",
            "metadata": {},
            "source": [
                "# Function to make project public - by default it is private and hidden from other users"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": 15,
            "metadata": {},
            "outputs": [],
            "source": [
                "def make_project_public(project_id):\n",
                "    requests.patch('http://' + server + '/SASEventStreamProcessingStudio/esp-project/'\n",
                "                   + project_id + '/authorization?private=false',\n",
                "                   headers={'Authorization': 'Bearer ' + access_token}, verify=False)"
            ]
        },
        {
            "cell_type": "markdown",
            "metadata": {},
            "source": [
                "# Function to create expected project model to be published"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": 25,
            "metadata": {},
            "outputs": [],
            "source": [
                "def create_publish_project_body(project_body, version):\n",
                "    project_body[\"name\"] = project_body[\"friendlyName\"]\n",
                "    project_body[\"description\"] = \"\"\n",
                "    project_body[\"friendlyName\"] = None\n",
                "    project_body[\"majorVersion\"] = str(version)\n",
                "    project_body[\"minorVersion\"] = \"0\"\n",
                "    project_body[\"version\"] = str(version) + '.0'\n",
                "    project_body[\"versionNotes\"] = \"notes\"\n",
                "    project_body[\"uploadedBy\"] = username\n",
                "    project_body[\"modifiedBy\"] = username\n",
                "    epoch_time = int(time.time())\n",
                "    project_body[\"uploaded\"] = epoch_time\n",
                "    project_body[\"modified\"] = epoch_time + 10\n",
                "    project_body[\"isDeployable\"] = False"
            ]
        },
        {
            "cell_type": "markdown",
            "metadata": {},
            "source": [
                "# Function to publish project"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": 26,
            "metadata": {},
            "outputs": [],
            "source": [
                "def publish_project(project_id, project_body, version):\n",
                "    folder_id = \"\"\n",
                "    create_publish_project_body(project_body, version)\n",
                "    publish_project_response = requests.post(\n",
                "        'http://' + server + '/SASEventStreamProcessingStudio/project-versions/projects/' + project_id,\n",
                "        data=json.dumps(project_body), headers=headers, verify=False)\n",
                "\n",
                "    if publish_project_response.status_code == 200:\n",
                "        folder_id = publish_project_response.json()[\"folderId\"]\n",
                "    else:\n",
                "        print('PUBLISH FAILED')\n",
                "        print(publish_project_response)\n",
                "        print('Version:' + str(version))\n",
                "    return folder_id"
            ]
        },
        {
            "cell_type": "markdown",
            "metadata": {},
            "source": [
                "# Function to synchronize projects from Studio to ESM"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": 27,
            "metadata": {},
            "outputs": [],
            "source": [
                "def synchronize_project_for_ESM(folder_id):\n",
                "    success = False\n",
                "    synchronize_project_response = requests.post(\n",
                "        'http://' + server + '/SASEventStreamProcessingStudio/project-versions/projects/synchronize/' + folder_id,\n",
                "        data=folder_id, headers=headers, verify=False)\n",
                "\n",
                "    if synchronize_project_response.status_code == 200:\n",
                "        success = True\n",
                "    return success"
            ]
        },
        {
            "cell_type": "markdown",
            "metadata": {},
            "source": [
                "# Function to get deployment details needed to start K8s cluster on ESM"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": 45,
            "metadata": {},
            "outputs": [],
            "source": [
                "def get_deployment_details():\n",
                "    success = True\n",
                "    deployment_id = ''\n",
                "    deployment_name = ''\n",
                "    projects_running_on_deployment = []\n",
                "    deployments_response = requests.get(\"http://\" + server + \"/SASEventStreamManager/deployment?noDetails=false\",\n",
                "                                        headers=headers, verify=False)\n",
                "\n",
                "    if deployments_response.status_code != 200:\n",
                "        print(\"Could not find any deployments\", deployments_response.text)\n",
                "        success = False\n",
                "\n",
                "    # Here we try to start clusters against the hard-coded deployment name\n",
                "    # if it does not exist, it will spin off cluster against 1st cluster\n",
                "    deployment_items = deployments_response.json()[\"items\"]\n",
                "    if len(deployment_items) > 0:\n",
                "        deployment_id = deployment_items[0][\"uuid\"]\n",
                "        deployment_name = deployment_items[0][\"label\"]\n",
                "        projects_running_on_deployment = get_project_names_from_deployment(deployment_items[0])\n",
                "        \n",
                "        for deployment in deployment_items:\n",
                "            if deployment[\"type\"] == \"cluster\" and deployment[\"label\"] == chosen_deployment_name:\n",
                "                deployment_id = deployment[\"uuid\"]\n",
                "                deployment_name = deployment[\"label\"]\n",
                "                projects_running_on_deployment = get_project_names_from_deployment(deployment)\n",
                "    else:\n",
                "        print(\"Could not find any deployments\", deployments_response.text)\n",
                "        success = False\n",
                "\n",
                "    return success, deployment_id, deployment_name, projects_running_on_deployment\n",
                "\n",
                "def get_project_names_from_deployment(deployment):\n",
                "    project_names = []\n",
                "    for server in deployment[\"servers\"]:\n",
                "        project_names = project_names + list(map(lambda project: project[\"name\"], server[\"projects\"]))\n",
                "    return project_names"
            ]
        },
        {
            "cell_type": "markdown",
            "metadata": {},
            "source": [
                "# Function to start K8s cluster on ESM"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": 48,
            "metadata": {},
            "outputs": [],
            "source": [
                "def start_k8s_cluster(k8s_project_body):\n",
                "    success, deployment_id, deployment_name, projects_running_on_deployment = get_deployment_details()\n",
                "\n",
                "    if k8s_project_body[\"name\"] in projects_running_on_deployment:\n",
                "        print(\"Project \" + k8s_project_body[\"name\"] + \" is already running on deployment \" + deployment_name +\".\")\n",
                "    elif success:\n",
                "        # Here are the deployment settings hard-coded for all projects\n",
                "        k8s_deployment_settings = {\n",
                "            \"persistentVolumeClaim\": \"sas-event-stream-processing-studio-app\",\n",
                "            \"requestsMemory\": \"1Gi\",\n",
                "            \"requestsCpu\": 1,\n",
                "            \"requestsGpu\": \"0\",\n",
                "            \"limitsMemory\": \"1Gi\",\n",
                "            \"limitsCpu\": 1,\n",
                "            \"limitsGpu\": \"0\",\n",
                "            \"minReplicas\": 1,\n",
                "            \"maxReplicas\": 1,\n",
                "            \"useLoadBalancer\": False,\n",
                "            \"loadBalancingPolicy\": \"none\",\n",
                "            \"averageUtilization\": \"50\",\n",
                "            \"loadBalancerTargetsList\": []\n",
                "        }\n",
                "        k8s_cluster_body = {\n",
                "            \"distinguisher\": deployment_name,\n",
                "            \"esmDeploymentId\": deployment_id,\n",
                "            \"gpuReliant\": False,\n",
                "            \"loadOnly\": False,\n",
                "            \"project\": k8s_project_body,\n",
                "            \"settings\": k8s_deployment_settings\n",
                "        }\n",
                "\n",
                "        response = requests.post(\"http://\" + server + \"/SASEventStreamManager/server/cluster\",\n",
                "                                 data=json.dumps(k8s_cluster_body), headers=headers, verify=False)\n",
                "        if response.status_code != 200:\n",
                "            print(\"error creating cluster\", response.text)\n",
                "        else:\n",
                "            print(\"Project \" + k8s_project_body[\"name\"], \" successfully started K8s pod in deployment: \" + deployment_name)"
            ]
        },
        {
            "cell_type": "markdown",
            "metadata": {},
            "source": [
                "# Create ESM Deployment for projects to run against"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": 58,
            "metadata": {},
            "outputs": [
                {
                    "name": "stdout",
                    "output_type": "stream",
                    "text": [
                        "Deployment test already exists, we can proceed to start the projects\n"
                    ]
                }
            ],
            "source": [
                "def create_ESM_cluster_deployment():\n",
                "    deployment_alread_exists_error = 'A deployment named \"' + chosen_deployment_name + '\" already exists'\n",
                "    deployment_body = {\n",
                "        \"name\": chosen_deployment_name,\n",
                "        \"type\": \"cluster\"\n",
                "    }\n",
                "    response = requests.post('http://' + server + '/SASEventStreamManager/deployment',\n",
                "                                            data=json.dumps(deployment_body), headers=headers, verify=False)\n",
                "    if deployment_alread_exists_error in response.text:\n",
                "        print (\"Deployment \" + chosen_deployment_name + \" already exists, we can proceed to start the projects\")\n",
                "    elif response.status_code != 201:\n",
                "        print(\"error creating ESM cluster deployment\", response.text)\n",
                "\n",
                "create_ESM_cluster_deployment()"
            ]
        },
        {
            "cell_type": "markdown",
            "metadata": {},
            "source": [
                "# Loop through all XML projects in xml_projects folder and do the following\n",
                "1. Check if projects have already been imported\n",
                "2. If yes, import it using next version number. Otherwise import it normally with version 1 \n",
                "3. Make project public so all users can see it\n",
                "4. Publish project\n",
                "5. Synchronize projects from Studio to ESM\n",
                "6. Create ESM Deployment Cluster on which to run the projects\n",
                "7. Start projets on K8s cluster on ESM"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": 50,
            "metadata": {},
            "outputs": [
                {
                    "name": "stdout",
                    "output_type": "stream",
                    "text": [
                        "\n",
                        "example_project is already imported\n",
                        "Creating new project version: 9\n",
                        "example_project.xml successfully published\n",
                        "Project example_project is already running on deployment test.\n",
                        "\n",
                        "star_wars is already imported\n",
                        "Creating new project version: 7\n",
                        "star_wars.xml successfully published\n",
                        "Project star_wars is already running on deployment test.\n"
                    ]
                }
            ],
            "source": [
                "def import_and_publish_xml_files():\n",
                "    current_dir = os.getcwd() + \"/xml_projects\"\n",
                "    projects = get_projects()\n",
                "    for file_name in os.listdir(current_dir):\n",
                "        print()\n",
                "        if not file_name.endswith('.xml'): continue\n",
                "        version = 1\n",
                "        project_id = None\n",
                "        xml_file_path = os.path.join(current_dir, file_name)\n",
                "        project_name = get_project_name_from_xml(xml_file_path)\n",
                "        data = open(xml_file_path, \"r\").read()\n",
                "        project_body = {'friendlyName': project_name, 'xml': data}\n",
                "\n",
                "        if projects:\n",
                "            project_that_matches_xml_name = None\n",
                "            for project in projects:\n",
                "                if project['friendlyName'] == project_name:\n",
                "                    project_that_matches_xml_name = project\n",
                "                    break\n",
                "\n",
                "            if project_that_matches_xml_name:\n",
                "                print(project_name + ' is already imported')\n",
                "                project_id = project_that_matches_xml_name['flowId']\n",
                "                version = get_next_project_version(project_id)\n",
                "                print('Creating new project version: ' + str(version))\n",
                "\n",
                "        if not bool(project_id):\n",
                "            print('Importing ' + project_name)\n",
                "            project_id = import_project_to_studio(project_body)\n",
                "            make_project_public(project_id)\n",
                "\n",
                "        folder_id = publish_project(project_id, project_body, version)\n",
                "\n",
                "        synchronize_project_for_ESM(folder_id)\n",
                "\n",
                "        if project_id:\n",
                "            print(file_name + ' successfully published', flush=True)\n",
                "        else:\n",
                "            print(file_name + ' failed to publish', flush=True)\n",
                "\n",
                "        k8s_project_body = {\"id\": project_id, \"name\": project_name, \"version\": version}\n",
                "        start_k8s_cluster(k8s_project_body)\n",
                "\n",
                "def get_project_name_from_xml(xml_file_path):\n",
                "    tree = ET.parse(xml_file_path)\n",
                "    root = tree.getroot()\n",
                "    project_name = root.attrib['name']\n",
                "    return project_name\n",
                "\n",
                "\n",
                "\n",
                "import_and_publish_xml_files()\n"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": null,
            "metadata": {},
            "outputs": [],
            "source": []
        }
    ],
    "metadata": {
        "kernelspec": {
            "display_name": "Python 3 (ipykernel)",
            "language": "python",
            "name": "python3"
        },
        "language_info": {
            "codemirror_mode": {
                "name": "ipython",
                "version": 3
            },
            "file_extension": ".py",
            "mimetype": "text/x-python",
            "name": "python",
            "nbconvert_exporter": "python",
            "pygments_lexer": "ipython3",
            "version": "3.11.2"
        }
    },
    "nbformat": 4,
    "nbformat_minor": 4
}